<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/image_type/cartoonify_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Determine if images are a cartoon or photograph
---
*Last Updated 7 September 2025*   
Classification accuracy for illustrated images and phylogenies was low for the trained model. This notebook uses an alternate approach that leverages image processing to identify images as photographic or non-photographic. First, cartoonify image, then compare change in color values. If change above a certain threshold, then image is likely photographic. If change below a certain threshold, image is likely non-photographic.
  
***Using 500 images from all image type classes, the best predictor of "not cartoon" was found to be Manhattan norm per pixel > 2.***

## Installs & Imports
---

In [ ]:
#@title Choose where to save results (or keep defaults) & set up environment
import os

# Use dropdown menu on right
save = "in Colab runtime (files deleted after each session)" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]
print("Saving results ", save)

# Mount google drive to export file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Type of classification pipeline
classif_type = "image_type" #@param ["image_type", "rating"] {allow-input: true}

# Type in the path to your working directory in form field to right
basewd = "/content/drive/MyDrive/train/tf2" #@param ["/content/drive/MyDrive/train/tf2"] {allow-input: true}
basewd = basewd + '/' + classif_type

# Folder where preprocessing outputs will be saved
folder = "pre-processing" # @param ["pre-processing","inspect_resul","results"] {"allow-input":true}
cwd = basewd + '/' + folder

# Folder where image metadata will be saved
data_folder = "image_data" #@param ["image_data"] {allow-input: true}
data_wd = cwd + '/' + data_folder

# Folder where train/test images will be saved
train_folder = "images" #@param ["images"] {allow-input: true}
train_wd = cwd + '/' + train_folder

# Enter image classes of interest in form field
filters = ["map", "phylo", "herb", "illus"] #@param ["[\"map\", \"phylo, \"herb\", \"illus\"]"] {type:"raw", allow-input: true}

# Download helper_funcs folder
!pip3 -q install --upgrade gdown
!gdown 1xmkrYEJKLJvei9q4zulKfqsGTgDvfvpR
!tar -xzvf helper_funcs.tar.gz -C .

# Install requirements.txt
!pip3 -q install -r requirements.txt

# Set up directory structure
from setup import setup_dirs

# Set up directory structure
setup_dirs(cwd, data_wd, train_wd)
print("\nWorking directory set to: \n", cwd)
print("\nImage metadata directory set to: \n", data_wd)
print("\nTraining images directory set to: \n", train_wd)

In [ ]:
#@title Import libraries

# For augmenting, displaying, and saving images
!pip install imaug
!pip install pillow

# For downloading images
!apt-get install aria2

# For importing/exporting files, working with arrays, etc
import pathlib
import os
import imageio
import time
import csv
import numpy as np
import pandas as pd
from urllib.request import urlopen
import mimetypes
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# For augmenting the images
import imgaug as ia
import imgaug.augmenters as iaa

# For drawing onto and plotting the images
import matplotlib.pyplot as plt
import cv2
import scipy
from scipy.linalg import norm
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# Define functions
from wrangle_data import *

# Image Type bundle urls
# Map, Herbarium Sheet, Phylogeny
bundles = ["https://editors.eol.org/other_files/bundle_images/classifier/maps.txt",
           "https://editors.eol.org/other_files/bundle_images/classifier/Phylogeny_images.txt",
           "https://editors.eol.org/other_files/bundle_images/classifier/herbarium_sheets_download.txt"]

# Illustration
# Pool zoology and botany into one illustration bundle
illus_bundles = ["https://editors.eol.org/other_files/bundle_images/classifier/Zoological_illustrations_download.txt",
                 "https://editors.eol.org/other_files/bundle_images/classifier/Botanical_illustrations_download.txt"]



# Set filename for saving classification results
def set_outfpath(imclass):
    outfpath = cwd + '/' + imclass + '_cartoonification_values.csv'
    print("\nSaving results to: \n", outfpath)

    return outfpath

# Export results
def export_results(results):
    results = pd.DataFrame(results)
    results = results.transpose()
    results.to_csv(outfpath, index=False, header=("filename", "mnorm", "mnorm_pp",
                                                 "znorm", "znorm_pp"))

# To save the figure
def save_figure(fig, imclass):
    figname = cwd + '/' + imclass + '_cartoonization_hists.png'
    fig.savefig(figname)
    print("Histograms saved to ", figname)

    return figname

## Download images to Google Drive from EOL, Wikimedia, and Flickr BHL image bundles
---
Run this step 5x (once per image bundle). For each iteration, use the dropdown menu to the right to select the image bundle to download images from.

In [ ]:
#@title Download images for each class

# Test pipeline with a smaller subset than 5k images?
run = "test with tiny subset" #@param ["test with tiny subset", "for all images"]
print("Run: ", run)

# Download images, augment them, and save to Google Drive
print("\nDownloading training images for each class")

# Download images for each class
for i, imclass in enumerate(filters):

        # Make folder for each class
        %cd $train_wd
        impath = train_wd + "/" + imclass + "/"
        if not os.path.isdir(impath):
            os.makedirs(impath)
        print("Path to images:")
        %cd $impath

        # Read in corresponding bundle
        # For map, herbarium sheet, phylogeny
        if imclass != 'illus':
            bundle = bundles[i]
            !wget --user-agent="Mozilla" $bundle
            print("Downloaded ", bundle)
            fn = os.path.basename(bundle)
            df = pd.read_table(fn)
            print(imclass, df.head())

        # For illustration
        else:
            il_fns = []
            for illus_bundle in illus_bundles:
                !wget --user-agent="Mozilla" $illus_bundle
                il_fn = os.path.basename(illus_bundle)
                il_fns.append(il_fn)

            df = pd.concat([pd.read_table(il_fn, header=None, names=['url']) for il_fn in il_fns], ignore_index=True)
            fn = 'illustrations_download.txt'
            print(imclass, df.head())

        # Take tiny subset or all images from bundle
        start, stop = set_start_stop(run, df)
        df = df.iloc[start:stop]
        df.to_csv(fn, index=False, header=False, lineterminator='\n')
        urls = df.iloc[:,0].dropna().astype(str).tolist()

        # Download images
        # Clean URLs list
        urls = [u.strip() for u in urls if u.strip() and u.startswith("http")]
        print(urls)
        download_images_parallel(urls, impath)

        # Check how many images downloaded
        print("Number of images downloaded to Google Drive for class {}:".format(imclass))
        !ls . | wc -l

        # Move image metadata text file(s) to image_data/bundles
        %cd $cwd
        impath = impath + "*.txt"
        !mv $impath image_data/

## Build "null" image class from EOL images
---   
Having a negative control will help train the classifier on what images do not belong in any of the above classes

In [ ]:
# Download null.zip images folder leftover from flower_fruit classifier model
%cd $train_wd
!pip3 install --upgrade gdown
!gdown 1-8-5EVq21jMUSvuEJynOBryKSJojOH49

# Unzip images
print("Unzipping botanical null images...")
!unzip null.zip

# Move unzipped null image folder content to images/null
# Google Drive Zipped folders have preserved directory structure
if not os.path.isdir('null'):
      os.makedirs('null')
# Only move 5 images if running in demo mode
if "tiny subset" in run:
    !shuf -n 6 -e content/drive/'My Drive'/summer20/classification/image_type/images/null/* | xargs -i mv {} null
# Run for all images
else:
    !mv content/drive/'My Drive'/summer20/classification/image_type/images/null/* null

# Check how many images in 'null/'
print("Number of images in 'null' class:")
%cd null
!ls . | wc -l

# Delete not needed files/folders
%cd ../
!rm -r content
!rm -r null.zip

# Add 'null' to imclasses list
filters.append('null')
print("Image classes to filter results by: ", filters)

## Cartoonify images
---   
Cartoonize images, then compare how different they are to the original and determine if they are likely a cartoon or photograph.

In [ ]:
#@title Cartoonify images

# Test pipeline with a smaller subset than 5k images?
run = "test with tiny subset" #@param ["test with tiny subset", "for 500 images"]
print("Run: ", run)

# For each image class, measure difference betweeen cartoonified and original
imclasses = filters
for imclass in imclasses:
    # Set filename for saving classification results
    outfpath = set_outfpath(imclass)
    # Make placeholder lists to record values for each image
    filenames, mnorms, mnorms_pp, znorms, znorms_pp = make_placeholders()
    # Get test images for cartoonizing
    df = get_test_images(imclass, cwd)

    # Cartoonify images
    try:
        start, stop = set_start_stop(run, df)
        for i, row in enumerate(df[start:stop], start=1):
            # Read in image from url or file
            im_path = row
            img = cv2.imread(im_path)

            # Cartoonization
            img2 = cartoonize(img)

            # Calculate differences between original and cartoonized image
            mnorm, mnorm_pp, znorm, znorm_pp = calc_img_diffs(img, img2)

            # Display cartoonized image when testing with tiny subset
            if "tiny subset" in run:
                display_images(img, img2, mnorm, mnorm_pp, znorm, znorm_pp)

            # Record results in placeholder lists to inspect results in next step
            results = record_results(row, mnorm, mnorm_pp, znorm, znorm_pp, filenames, mnorms, mnorms_pp, znorms, znorms_pp)
    except:
        pass

    # Combine to df and export results
    export_results(results)

In [ ]:
#@title Combine model outputs for image type classes
%cd $cwd

# Get cartoonization files for each image type class
imclasses = filters
all_filenames = [imclass + '_cartoonification_values.csv' for imclass in imclasses]
imclass_dict = dict(zip(all_filenames, imclasses))
# Set threshold for cartoon or not
thresh = 2 #@param {type:"number"}

# Loop through cartoonization files and display histograms
for fn in all_filenames:
    print("\nInspecting cartoonization values for: ", fn)
    imclass = imclass_dict[fn]
    df = pd.read_csv(fn, header=0)
    mnorms = df['mnorm']
    mnorms_pp = df['mnorm_pp']
    znorms = df['znorm']
    znorms_pp = df['znorm_pp']

    # Plot parameters
    kwargs = dict(alpha=0.5, bins=15)
    fig, (a, b, c, d) = plt.subplots(4, figsize=(10, 10), sharey=True, constrained_layout=True)
    fig.suptitle('{}: Image differences after cartoonization (n={} imgs)'.format(imclass, len(df)))

    # Manhattan norm values
    bins, counts = np.histogram(mnorms)
    a.hist(mnorms, color='y', label='True Det', **kwargs)
    a.set_title("{}: Manhattan norm".format(imclass));

    # Zero norm values
    bins, counts = np.histogram(znorms)
    c.hist(znorms, color='y', label='True Det', **kwargs)
    c.set_title("{}: Zero norm".format(imclass));

    # Manhattan norm values per pixel
    bins, counts = np.histogram(mnorms_pp)
    b.hist(mnorms_pp, color='y', label='True Det', **kwargs)
    b.set_title("{}: Manhattan norm per pixel".format(imclass));

    # Zero norm values per pixel
    bins, counts = np.histogram(znorms_pp)
    d.hist(znorms_pp, color='y', label='True Det', **kwargs)
    d.set_title("{}: Zero norm per pixel".format(imclass));

    # Add Y-axis labels
    for ax in fig.get_axes():
        ax.set(ylabel='Freq (# imgs)')
        if thresh:
            ax.axvline(thresh, color='k', linestyle='dashed', linewidth=1)

    # Export histograms
    figname = save_figure(fig, imclass)